## Classification of MNIST dataset with tensor networks

In [80]:
# import packages here
# --------------------------------------
import matplotlib.pyplot as plt

import qtealeaves as qtl
from qtealeaves.convergence_parameters.conv_params import TNConvergenceParameters
from qtealeaves.emulator.mps_simulator import MPS
from qtealeaves.emulator import StateVector

from tensorflow.keras.datasets import mnist
import numpy as np


### Load MNIST dataset

In [81]:
# Load dataset here
# --------------------------------------
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)


(60000, 28, 28)
(10000, 28, 28)


In [82]:
# Selecting only images with the numbers 3 and 8
train_filter = np.where((y_train == 3 ) | (y_train == 8))
test_filter = np.where((y_test == 3) | (y_test == 8))

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Reduce amount of images
nr_images = 500

x_train, y_train, x_test, y_test = x_train[:nr_images, :, :], y_train[:nr_images], x_test[:nr_images, :, :], y_test[:nr_images]

x_train = x_train / 255
x_test = x_test / 255

In [83]:
import skimage.measure

x_train = skimage.measure.block_reduce(x_train, (1,6,6), np.mean)
x_test = skimage.measure.block_reduce(x_test, (1,6,6), np.mean)



In [84]:
# Flatten image
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]*x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]*x_test.shape[2]))

x_test.shape

(500, 25)

In [85]:
# Create state vector with encoding 2
num_sites = x_train[1]
state = x_train
state_vector = StateVector(num_sites, state=state)

NameError: name 'StateVector' is not defined

### Build classifier


In [23]:
# Convert dataset into a list of MPS
# --------------------------------------
X_train_mps = MPS.from_tensor_list(x_train, conv_params=TNConvergenceParameters)
X_test_mps = MPS.from_tensor_list(x_test, conv_params=TNConvergenceParameters)



IndexError: tuple index out of range

In [21]:
# Optimize MPS

# get number of sites
num_sites = x_train.shape[1]*x_train.shape[2]

# define max bond dimension
max_bond = 10

# define batch size, learning rate and number of sweeps
batch_size = 256
learning_rate = 1e-4
num_sweeps = 10     # epochs

# initialize MPS for the classifier
conv_params = TNConvergenceParameters(
                    max_bond_dimension=max_bond)
tn_classifier = MPS(num_sites, conv_params, dtype=float)

svd, loss = tn_classifier.ml_optimize_mps(X_train_mps,
                                y_train,
                                batch_size=batch_size,
                                learning_rate=learning_rate,
                                num_sweeps=num_sweeps,
                                n_jobs=1,
                                verbose=True)

==================== Sweep 0 started ====================


TypeError: Only two MPS classes can be contracted

### Prediction on train and test dataset

In [ ]:
# Predict labels with the trained MPS
y_train_pred = tn_classifier.ml_predict(X_train_mps, n_jobs=1)
y_test_pred = tn_classifier.ml_predict(X_test_mps, n_jobs=1)

In [ ]:
# Get accuracy
accuracy_train = ...
print(accuracy_train)

accuracy_test = ...
print(accuracy_test)

# 